<a href="https://colab.research.google.com/github/Angel-Diaz-H/scripts-python-sql/blob/main/sql_scripts_respaldos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re

archivo_entrada = "/content/file3.sql"
archivo_salida = "backup.sql"

#leer archivo
with open(archivo_entrada, "r") as f:
    contenido = f.read()

# Dividir el archivo en bloques por cada sentencia que termina con ';'
sentencias = re.split(r';\s*\n?', contenido, flags=re.MULTILINE)

selects_generados = []

for sentencia in sentencias:
    sentencia = sentencia.strip()
    #ignorar y saltar comentarios
    if not sentencia or sentencia.startswith("--"):
        continue

    #delete
    if sentencia.upper().startswith("DELETE FROM"):
        #'DELETE' por 'SELECT *'
        linea_select = re.sub(r"DELETE FROM\s+", "SELECT * FROM ", sentencia, flags=re.IGNORECASE)
        selects_generados.append(linea_select + ";")  #Agregar ;

    #UPDATE
    elif sentencia.upper().startswith("UPDATE"):
        #search tabla después de UPDATE
        tabla_match = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
        if tabla_match:
            tabla = tabla_match.group(1)
            #WHERE (en todo el bloque)
            where_idx = sentencia.upper().find("WHERE")
            if where_idx != -1:
                condicion = sentencia[where_idx:]
                linea_select = f"SELECT * FROM {tabla} {condicion}"
                selects_generados.append(linea_select + ";")

#save
with open(archivo_salida, "w") as f:
    for sel in selects_generados:
        f.write(sel + "\n")

print(f"{len(selects_generados)} SELECTs respaldados en '{archivo_salida}'")


In [10]:
import re

archivo_entrada = "/content/file4.sql"
archivo_salida = "backup_agrupado.sql" # Nombre del archivo de salida

# leer archivo
with open(archivo_entrada, "r") as f:
    contenido = f.read()

# Dividir el archivo en bloques por cada sentencia que termina con ';'
sentencias = re.split(r';\s*\n?', contenido, flags=re.MULTILINE)

# Diccionario para agrupar condiciones por tabla y columna
condiciones_agrupadas = {} # Formato: {('tabla', 'columna'): [valor1, valor2, ...]}

for sentencia in sentencias:
    sentencia = sentencia.strip()
    # ignorar y saltar comentarios
    if not sentencia or sentencia.startswith("--"):
        continue

    # DELETE
    if sentencia.upper().startswith("DELETE FROM"):
        # Extraer tabla
        tabla_match = re.match(r"DELETE FROM\s+([^\s]+)", sentencia, re.IGNORECASE)
        if tabla_match:
            tabla = tabla_match.group(1)
            # Extraer la condición WHERE
            where_idx = sentencia.upper().find("WHERE")
            if where_idx != -1:
                condicion_completa = sentencia[where_idx + len("WHERE"):].strip()

                # Extraer la columna y el valor para la condición IN
                match_in = re.match(r"(\w+)\s*=\s*(.*)", condicion_completa)
                if match_in:
                    columna = match_in.group(1).strip()
                    valor = match_in.group(2).strip()

                    # Agrupar la condición
                    if (tabla, columna) not in condiciones_agrupadas:
                        condiciones_agrupadas[(tabla, columna)] = []
                    # Asegurarse de no agregar valores duplicados si eso es deseado
                    if valor not in condiciones_agrupadas[(tabla, columna)]:
                         condiciones_agrupadas[(tabla, columna)].append(valor)

    # UPDATE (mantener lógica existente si la necesitas, o adaptar para agrupar si aplica)
    elif sentencia.upper().startswith("UPDATE"):
        # search tabla después de UPDATE
        tabla_match = re.match(r"UPDATE\s+([^\s]+)", sentencia, re.IGNORECASE)
        if tabla_match:
            tabla = tabla_match.group(1)
            # WHERE (en todo el bloque)
            where_idx = sentencia.upper().find("WHERE")
            if where_idx != -1:
                condicion = sentencia[where_idx:]
                linea_select = f"SELECT * FROM {tabla} {condicion}"
                # Aquí podrías decidir si quieres agrupar UPDATEs también,
                # o mantenerlos separados como en tu código original.
                # Para este ejemplo, los mantenemos separados y no los incluimos en la salida agrupada de DELETEs.
                # Si quieres incluir SELECTs de UPDATEs, necesitarías otro mecanismo de almacenamiento.


# Generar sentencias DELETE agrupadas (o SELECT, si ese es tu objetivo final)
sentencias_agrupadas_generadas = []
for (tabla, columna), valores in condiciones_agrupadas.items():
    if valores: # Solo generar si hay valores para la condición IN
        # Formatear los valores para la cláusula IN
        valores_formateados = ", ".join(valores)
        # Generar la sentencia DELETE agrupada
        # Si tu objetivo es generar SELECTs para backup, cambia "DELETE FROM" a "SELECT *"
        linea_agrupada = f"DELETE FROM {tabla} WHERE {columna} IN ({valores_formateados});"
        sentencias_agrupadas_generadas.append(linea_agrupada)

# save
with open(archivo_salida, "w") as f:
    for sel in sentencias_agrupadas_generadas:
        f.write(sel + "\n")

print(f"{len(sentencias_agrupadas_generadas)} sentencias respaldadas (agrupadas) en '{archivo_salida}'")

2 sentencias respaldadas (agrupadas) en 'backup_agrupado.sql'
